# Fragment Design: Generate New Molecules

-  In this notebook, we will generate new molecules using the fragment design approach. We will start by importing the necessary libraries and loading the dataset.

## 检测 GPU 是否可用

In [1]:
# 检测是否GPU可用
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


2024-10-18 11:27:27.913404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Please install GPU version of TF


In [1]:
from pathlib import Path
import os

# 获取当前工作目录
HERE = Path(os.getcwd())
DATA = HERE / 'data'
if not DATA.exists():
    DATA.mkdir(parents=True, exist_ok=True)
print(DATA)

/Users/wangyang/Desktop/AI-drug-design/list/05_workshop/03_Fragment-Design_Generate-New-Molecules/data


## Import Libraries


In [ ]:
# 导入必要的库
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit import RDLogger
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import BRICS
from rdkit.Chem import AllChem
from rdkit.Chem import rdChemReactions
from rdkit.Chem import Descriptors
from rdkit.Chem.FilterCatalog import FilterCatalogParams, FilterCatalog
from rdkit.Chem import rdMolDescriptors
import random
import os
import itertools
from pathlib import Path
import os

In [ ]:
# 获取当前工作目录
HERE = Path(os.getcwd())
DATA = HERE / 'data'
if not DATA.exists():
    DATA.mkdir(parents=True, exist_ok=True)
print(DATA)
# 禁用 RDKit 日志
RDLogger.DisableLog('rdApp.*')

In [ ]:
# 设置随机种子
random.seed(42)

In [ ]:
# 1. 数据加载
df = pd.read_csv(DATA / 'egfr_data_ubstructures_matches.csv')
df = df[df['smiles'].notnull()]

In [ ]:
df.head()

In [ ]:
smiles_list = df['smiles'].tolist()
molecules = [Chem.MolFromSmiles(smile) for smile in smiles_list]
molecules = [mol for mol in molecules if mol is not None]
print(f'有效分子数量: {len(molecules)}')

In [ ]:
# 2. 片段提取
# 使用 Murcko Scaffold 提取骨架
scaffolds = set()
for mol in molecules:
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    if scaffold:
        scaffolds.add(Chem.MolToSmiles(scaffold))

print(f'提取到的独特 Murcko 骨架数量: {len(scaffolds)}')

In [ ]:
# 使用 BRICS 方法提取片段
brics_fragments = set()
for mol in molecules:
    frags = BRICS.BRICSDecompose(mol)
    brics_fragments.update(frags)

print(f'提取到的独特 BRICS 片段数量: {len(brics_fragments)}')


In [ ]:
# 合并所有片段
all_fragments_smiles = list(scaffolds.union(brics_fragments))

# 将片段转换为 RDKit 分子对象
fragment_mols = [Chem.MolFromSmiles(frag) for frag in all_fragments_smiles]
fragment_mols = [frag for frag in fragment_mols if frag is not None]

print(f'总片段数量（去重后）: {len(fragment_mols)}')

In [ ]:
# 3. 修正后的反应 SMARTS 集合
reaction_smarts = [
    # 酯化反应
    '[C:1](=[O])[O][H].[O:2][C:3]>>[C:1](=[O])[O:2][C:3]',
    # 酰胺化反应
    '[C:1](=[O])[O][H].[N:2][H][H]>>[C:1](=[O])[N:2][H]',
    # 醚化反应
    '[O:1][H].[C:2][Br]>>[O:1][C:2]',
    # 胺化反应
    '[N:1][H][H].[C:2][Br]>>[N:1][C:2]',
    # Suzuki 偶联反应
    '[B:1]([O])[O].[C:2][Br]>>[C:2][B:1]',
    # Heck 反应
    '[C:1]=[C:2].[C:3]=[C:4][Br]>>[C:1]=[C:2][C:3]=[C:4]',
    # Sonogashira 反应
    '[C:1]#[C:2][H].[C:3][Br]>>[C:1]#[C:2][C:3]',
    # Mitsunobu 反应
    '[O:1][H].[N:2][H][C:3]>>[N:2][C:3]',
    # Aldol 缩合反应
    '[C:1]=O.[C:2]=O>>[C:1](O)[C:2]=O',
    # Knoevenagel 缩合反应
    '[C:1]=O.[C:2]=C[H]>>[C:1]=C[C:2]=C',
    # Diels-Alder 反应
    '[C:1]=[C:2].[C:3]=[C:4]>>[C:1]1[C:2][C:3][C:4]1',
    # 取代反应（卤代烃与醇）
    '[C:1][Cl].[O:2][H]>>[C:1][O:2]',
    # 取代反应（卤代烃与硫醇）
    '[C:1][Br].[S:2][H]>>[C:1][S:2]',
    # 还原胺化
    '[C:1]=O.[N:2][H][H]>>[C:1][N:2][H]',
    # Wittig 反应
    '[C:1]=O.[P:2][C:3][C:4]>>[C:1]=[C:3]',
    # Michael 加成
    '[C:1]=[C:2][C:3]=O.[N:4][H][H]>>[C:1][C:2][C:3](O)[N:4][H]',
    # Friedel-Crafts 烷基化
    '[c:1].[C:2][Cl]>>[c:1][C:2]',
    # Friedel-Crafts 酰基化
    '[c:1].[C:2](=O)[Cl]>>[c:1][C:2]=O',
    # 氯化反应
    '[C:1][H].[Cl][Cl]>>[C:1][Cl]',
    # 溴化反应
    '[C:1][H].[Br][Br]>>[C:1][Br]',
    # 硝化反应
    '[c:1].[N+](=O)[O-]>>[c:1][N+](=O)[O-]',
    # 还原反应（醛到醇）
    '[C:1]=O>>[C:1][O][H]',
    # 氧化反应（醇到醛）
    '[C:1][O][H]>>[C:1]=O',
    # 氨解反应
    '[C:1](=O)[O][C:2]>>[C:1](=O)[N][H]',
    # 成环反应
    '[C:1][C:2].[C:3][C:4]>>[C:1][C:2][C:3][C:4]',
    # Grignard 反应
    '[C:1][Mg][Br].[C:2]=O>>[C:1][C:2][O][H]',
    # 保护基引入（甲基化）
    '[O:1][H].[C:2][I]>>[O:1][C:2]',
    # 保护基去除（脱甲基）
    '[O:1][C:2][H]>>[O:1][H]',
    # 甲酰化反应
    '[C:1][H].[C:2](=O)[Cl]>>[C:1][C:2]=O',
    # 脱水反应
    '[C:1][O][H].[C:2][O][H]>>[C:1]=[C:2]',
    # 氧化反应（伯醇到羧酸）
    '[C:1][O][H]>>[C:1](=O)[O][H]',
    # Mannich 反应
    '[C:1]=O.[C:2]=C.[N:3][H][H]>>[C:1][C:2][N:3][H]',
    # Beckmann 重排
    '[C:1](=O)[N:2][OH]>>[C:1][N:2]=O',
    # Claisen 缩合
    '[C:1](=O)[O][C:2].[C:3]=O>>[C:1](=O)[C:3]=O',
    # Gabriel 合成
    '[C:1][Br].[N:2][H]>>[C:1][N:2][H]',
    # Sandmeyer 反应
    '[C:1][N+][N-].[Cu][Cl]>>[C:1][Cl]',
    # Baeyer-Villiger 氧化
    '[C:1](=O)[C:2]>>[C:1](=O)[O][C:2]',
    # 脱卤反应
    '[C:1][Br]>>[C:1][H]',
    # 酰氯合成
    '[C:1](=O)[O][H].[Cl][Cl]>>[C:1](=O)[Cl]',
    # 二硫键形成
    '[S:1][H].[S:2][H]>>[S:1][S:2]',
    # 叠氮化反应
    '[C:1][Br].[N3]>>[C:1][N3]',
    # 环氧化反应
    '[C:1]=[C:2]>>[C:1]1[O][C:2]1',
    # Ozonolysis
    '[C:1]=[C:2]>>[C:1](=O).[C:2](=O)',
    # Ester Hydrolysis
    '[C:1](=O)[O][C:2].[H][O][H]>>[C:1](=O)[O][H].[C:2][O][H]',
    # Amide Hydrolysis
    '[C:1](=O)[N][C:2].[H][O][H]>>[C:1](=O)[O][H].[N][C:2]',
    # Wittig-Horner 反应
    '[C:1]=O.[P:2](=O)[C:3][C:4]>>[C:1]=[C:3]',
    # Enamine 反应
    '[C:1]=O.[N:2][H][C:3]>>[C:1]=[N:2][C:3]',
    # Curtius 重排
    '[C:1](=O)[N3]>>[C:1][N][H]',
    # Ullmann 反应
    '[C:1][Br].[C:2][Br]>>[C:1][C:2]',
    # Hantzsch 合成
    '[C:1]=O.[C:2]=O.[N:3][H][H]>>[C:1][C:2][N:3]',
    # Pechmann 反应
    '[C:1]=O.[C:2]=C[O][H]>>[C:1][C:2]=O',
    # Skraup 合成
    '[C:1]=C.[N:2][H][H].[O]>>[C:1][N:2]=C',
    # Tiffeneau-Demjanov 重排
    '[C:1][C:2][N][H][H]>>[C:1]=O.[C:2][H]'
]


reactions = [rdChemReactions.ReactionFromSmarts(rs) for rs in reaction_smarts]



In [ ]:

# 4. 使用笛卡尔积生成片段组合并生成新分子
def generate_new_molecules(fragments, reactions, num_molecules=1000):
    new_molecules = set()
    fragment_pairs = list(itertools.product(fragments, repeat=2))
    random.shuffle(fragment_pairs)  # 随机打乱组合顺序
    max_attempts = len(fragment_pairs)
    attempts = 0

    for frag1, frag2 in fragment_pairs:
        if attempts >= max_attempts or len(new_molecules) >= num_molecules:
            break
        attempts += 1

        for rxn in reactions:
            try:
                # 检查反应物是否匹配反应模式
                reactants = []
                if rxn.IsMoleculeReactant(frag1):
                    reactants.append(frag1)
                if rxn.IsMoleculeReactant(frag2):
                    reactants.append(frag2)
                if len(reactants) != rxn.GetNumReactantTemplates():
                    continue

                products = rxn.RunReactants(reactants)

                # 提取生成的产品
                for product in products:
                    mol = product[0]
                    Chem.SanitizeMol(mol)
                    smi = Chem.MolToSmiles(mol)
                    new_molecules.add(smi)
            except Exception as e:
                continue

            if len(new_molecules) >= num_molecules:
                break

    return list(new_molecules)


In [ ]:
# 生成新分子
print("开始生成新分子...")
new_smiles = generate_new_molecules(fragment_mols, reactions, num_molecules=1000)
print(f'生成的新分子数量: {len(new_smiles)}')


In [ ]:

# 将 SMILES 转换为分子对象
new_mols = [Chem.MolFromSmiles(smi) for smi in new_smiles]
new_mols = [mol for mol in new_mols if mol is not None]


In [ ]:
# 5. 验证新分子
# 应用 Lipinski 规则和 PAINS 过滤器
def filter_molecules(mols):
    filtered_mols = []
    # 初始化 PAINS 过滤器
    params = FilterCatalogParams()
    params.AddCatalog(FilterCatalogParams.FilterCatalogs.PAINS)
    catalog = FilterCatalog(params)

    for mol in mols:
        mol_weight = Descriptors.MolWt(mol)
        logp = Descriptors.MolLogP(mol)
        num_h_donors = rdMolDescriptors.CalcNumHBD(mol)
        num_h_acceptors = rdMolDescriptors.CalcNumHBA(mol)
        rotatable_bonds = Descriptors.NumRotatableBonds(mol)

        # 应用 Lipinski 规则
        lipinski_pass = (mol_weight <= 500 and
                         logp <= 5 and
                         num_h_donors <= 5 and
                         num_h_acceptors <= 10 and
                         rotatable_bonds <= 10)

        # 应用 PAINS 过滤器
        pains_matches = catalog.GetMatches(mol)
        pains_pass = len(pains_matches) == 0

        if lipinski_pass and pains_pass:
            filtered_mols.append(mol)

    return filtered_mols


In [ ]:
valid_mols = filter_molecules(new_mols)
print(f'符合筛选条件的分子数量: {len(valid_mols)}')

In [ ]:
# 6. 可视化生成的分子
# 绘制生成的分子（前16个）
if valid_mols:
    img = Draw.MolsToGridImage(valid_mols[:16], molsPerRow=4, subImgSize=(200, 200))
    img.show()
else:
    print("没有符合条件的分子。")

In [ ]:
# 7. 保存结果
# 创建 data 目录（如果不存在）
os.makedirs('data', exist_ok=True)

In [ ]:
# 将有效的分子 SMILES 保存到 CSV 文件
valid_smiles = [Chem.MolToSmiles(mol) for mol in valid_mols]
output_df = pd.DataFrame(valid_smiles, columns=['SMILES'])
output_df.to_csv(DATA / 'Fragment-Design_Generate-New-Molecules.csv', index=False)
print("生成的分子已保存到 data/Fragment-Design_Generate-New-Molecules.csv 文件中。")